# This Notebook Trains the Anisotropic Classifier

The anistotropic version of the classifier uses alternating horizontal and vertical 1D convolutions instead of 2D convolutions in the decoding phase of a segnet. 

- The notebook that modifies the _facade-segnet_ net is [modify-segnet-for-anisotropy.ipynb](anisotropic-training/modify-segnet-for-anisotropy.ipynb)
- Training can also be accomplished modifying paths as needed and then running the shell scripts:
    - [scripts/anisotropic-training/start-training.sh](anisotropic-training/start-training.sh)
    - [scripts/anisotropic-training/resume-training.sh](anisotropic-training/resume-training.sh)
    - [scripts/anisotropic-training/finish-training.sh](anisotropic-training/finish-training.sh)

> **NOTE** When trianing I have my data in an external, larger hard drive. This gives me enough storage to hold many snapshots of the classifier as well as preprocessed imagery for traiing. I use a soft-link so that the extranl drive appears to be part of the project. 

---

>**NOTE:** You may need to close any kernel / delete anty nets from other noteboosk that are using the GPU, this is quite memory intensive.

# Start The Engines....

In [ ]:
%pylab notebook

In [ ]:
import caffe
caffe.set_device(0)
caffe.set_mode_gpu()

I prefer to list paths relative to the root of the project, even though this notebook in in a 'scripts' folder. 

In [ ]:
cd ../..

During training, I use some python modules that are in the scripts folder. 

In [ ]:
import os
sys.path.insert(0, os.path.abspath('./scripts/anisotropic-training'))

Training takes a long time, but I like to know ASAP when something fails. 
These cells let me send myself an email when things go wrong.

# Put on your seatbelts ....

In [ ]:
import getpass
pwd = getpass.getpass('gmail:')

In [ ]:
import smtplib

def tell_me_about(issue):
    email_address = "femianjc@miamioh.edu"

    server = smtplib.SMTP('smtp.gmail.com', 587)
    server.starttls()
    server.login(email_address, pwd)

   
    msg = """\
FROM: "hal.csi.miamioh.edu" <femianjc@miamioh.edu>
SUBJECT: Deep Learning Alert

{}
.
""".format(str(issue))

    server.sendmail(email_address, email_address, msg)
    server.quit()

# Load The Solver (Resuming from Last Session)

Load the solver, attemptiong to resume from the last saved snapshot.

In [ ]:
import re
solver_proto = open('scripts/anisotropic-training/solver.prototxt').read()
snapshot_prefix = re.search('^snapshot_prefix *: &*\"(.*)\"', solver_proto, re.MULTILINE).group(1).strip(' "')
print snapshot_prefix

In [ ]:
def get_iter(path):
    s = os.path.splitext(path)[0]
    s = s.rsplit('_')[-1]
    iter = int(s)
    return iter


def get_last_iter(names):
    iters = [get_iter(name) for name in names]
    max_iter = max(iters) if len(iters) > 0 else ''
    return max_iter

from glob import glob
print  get_last_iter(glob(snapshot_prefix + '*.caffemodel'))

In [ ]:
%pushd scripts/anisotropic-training
%pwd
import warnings

try:
    del net  # Delete the old net if it alrady exists
except NameError:
    pass

with warnings.catch_warnings():
    warnings.filterwarnings(action='ignore')  # Ignore warning about mpl.use
    solver = caffe.get_solver('solver.prototxt')
%popd

In [ ]:
weights = '/home/shared/Projects/Facades/mybook/anisotropic/resume-from-round-one.caffemodel'
solver.net.copy_from(weights)

#  Configure the data layer 

I want to make sure it does nto clutter this notebook with extra output

In [ ]:
data_layer = solver.net.layers[0]

In [ ]:
data_layer.verbose = False

In [ ]:
print "Training", len(solver.net.outputs), "classes:\n\t-", '\n\t- '.join(solver.net.outputs)

In [ ]:
epoch_size = len(data_layer.files)
batch_size = data_layer.batch_size
iters_per_epoch = int(ceil(epoch_size/batch_size))
print "{} samples per epoch".format(epoch_size)
print "{} samples per (training) batch".format(batch_size)
print "{} iterations per epoch".format(iters_per_epoch)


# Start the main training loop!

In [ ]:
from pyfacades.util import softmax
import pyfacades.models.independant_12_layers.caffe_layers
from pyfacades.util import channels_last
import traceback

In [ ]:
losses = {key:[] for key in solver.net.outputs}  
losses['total'] = []

In [ ]:
fig = figure(figsize(10, 10))
ncols = 2
nrows = int(ceil(len(solver.net.outputs)/float(ncols)))
axes = {key:subplot(nrows, ncols, i+1) for i, key in enumerate(solver.net.outputs)}

try:
    while True:
        solver.step(1)

        assert not isnan(solver.net.params['conv1_1'][0].data.var())
        clf()

        losses['total'].append(0)
        for output in solver.net.outputs:
            losses[output].append(float(solver.net.blobs[output].data))
            losses['total'][-1] += losses[output][-1]
        
        if solver.iter % 10 != 0:
            continue

        subplot(2, 1, 1)  
        cla()
        title("Tot: {:2.4f}, Win {:2.4}, Iter {}, Epoch {}".format( 
                losses['total'][-1], losses['window-loss'][-1], solver.iter, data_layer.epochs))
        for output in ('window-loss',): #solver.net.outputs:
            xmax = len(losses['total'])
            xmin = max(0, xmax-100)
            plot(arange(xmin, xmax), losses[output][xmin:xmax])
        plot(arange(xmin, xmax), losses['total'][xmin:xmax])

        subplot(2, 2, 3)
        cla()
        imshow(channels_last(solver.net.blobs['data'].data[0])/255.)
        imshow(solver.net.blobs['facade'].data[0,0], alpha=0.3, cmap=cm.Reds)
        imshow(solver.net.blobs['window'].data[0,0], alpha=0.3, cmap=cm.Greens)
        subplot(2, 2, 4)
        cla()
        imshow(softmax(solver.net.blobs['conv-window'].data[0][(0,2,3),...])[1], cmap=cm.gray)
        imshow(solver.net.blobs['facade'].data[0,0], alpha=0.3, cmap=cm.Reds)
        imshow(solver.net.blobs['window'].data[0,0], alpha=0.3, cmap=cm.Greens)

        fig.tight_layout()
        fig.canvas.draw()
except Exception as e:
    msg = traceback.format_exc()
    print msg
    tell_me_about(msg)